# Premier League 2017-18 데이터와 VAEP 데이터 통합

이 노트북은 Premier League 2017-18 시즌 통계 데이터와 VAEP 모델 결과를 통합합니다.

## 데이터 소스
- Premier League 데이터: `data/player_match_stats/ENG-Premier League-2017-18.csv`
- VAEP 데이터: `data/vaep_results/player_season_vaep_atomic.csv`
- 선수별 리그 정보: `data/vaep_results/player_game_vaep_atomic.csv` + `data/wyscout/matches_*.json`
- 선수 정보: `data/wyscout/players.json`

In [14]:
import pandas as pd
import json
from pathlib import Path
import re

# 파일 경로 설정
stats_dir = Path('../data/player_match_stats')
vaep_file = Path('../data/vaep_results/player_season_vaep_atomic.csv')
player_game_vaep_file = Path('../data/vaep_results/player_game_vaep_atomic.csv')
players_json = Path('../data/wyscout/players.json')
matches_dir = Path('../data/wyscout')
competitions_json = Path('../data/wyscout/competitions.json')
output_file = Path('../data/player_season_stats_with_vaep_all_leagues_2017-18.csv')

# 원본 리그 파일 목록 (모든 통계 포함)
league_files = [
    'ENG-Premier League-2017-18.csv',
    'ESP-La Liga-2017-18.csv',
    'FRA-Ligue 1-2017-18.csv',
    'GER-Bundesliga-2017-18.csv',
    'ITA-Serie A-2017-18.csv'
]

## 1. 모든 리그 데이터 로드

In [15]:
# 모든 리그 데이터 로드 (원본 파일, 모든 통계 포함)
print("모든 리그 데이터 로드 중...")
all_leagues_df = []

# 컬럼명 매핑 (원본 파일의 실제 컬럼명 -> 최종 컬럼명)
# 원본 파일은 header=2로 읽으면 첫 번째 행이 실제 컬럼명
column_mapping = {
    # 기본 정보 (이미 올바른 이름)
    'league': 'league',
    'season': 'season',
    'game': 'game',
    'team': 'team',
    'player': 'player',
    'jersey_number': 'jersey_number',
    'nation': 'nation',
    'pos': 'pos',
    'age': 'age',
    # Performance 지표
    'min': 'minutes',
    'Gls': 'goals',
    'Ast': 'assists',
    'PK': 'penalties',
    'PKatt': 'penalty_attempts',
    'Sh': 'shots',
    'SoT': 'shots_on_target',
    'CrdY': 'yellow_cards',
    'CrdR': 'red_cards',
    'Touches': 'touches',
    'Tkl': 'tackles',
    'Int': 'interceptions',
    'Blocks': 'blocks',
    # Expected 지표
    'xG': 'xG',
    'npxG': 'npxG',
    'xAG': 'xAG',
    # SCA/GCA
    'SCA': 'SCA',
    'GCA': 'GCA',
    # Passes
    'Cmp': 'passes_completed',
    'Att': 'passes_attempted',
    'Cmp%': 'pass_completion_pct',
    'PrgP': 'progressive_passes',
    # Carries
    'Carries': 'carries',
    'PrgC': 'progressive_carries',
    # Take-Ons (Att는 이미 passes_attempted로 매핑되므로 별도 처리 필요)
    'Succ': 'take_ons_successful'
}

for league_file in league_files:
    file_path = stats_dir / league_file
    if file_path.exists():
        print(f"  로드 중: {league_file}")
        try:
            # header=[0, 1]로 읽기 (multi-level header)
            df = pd.read_csv(file_path, header=[0, 1])
            
            # 컬럼명 매핑 (multi-level header 처리)
            rename_dict = {}
            col_list = list(df.columns)
            
            for i, col in enumerate(col_list):
                if isinstance(col, tuple):
                    # Unnamed 컬럼 처리: 첫 번째 5개 컬럼은 league, season, game, team, player
                    if i < 5 and str(col[1]).startswith('Unnamed'):
                        base_cols = ['league', 'season', 'game', 'team', 'player']
                        rename_dict[col] = base_cols[i]
                    # 두 번째 레벨 이름 사용 (실제 컬럼명)
                    elif col[1] and not str(col[1]).startswith('Unnamed'):
                        actual_name = col[1]
                        # 매핑 딕셔너리에 있으면 사용
                        if actual_name in column_mapping:
                            rename_dict[col] = column_mapping[actual_name]
                        else:
                            rename_dict[col] = actual_name
                    # 첫 번째 레벨 이름 사용
                    elif col[0] and not str(col[0]).startswith('Unnamed'):
                        if col[0] in column_mapping:
                            rename_dict[col] = column_mapping[col[0]]
                        else:
                            rename_dict[col] = col[0]
                    else:
                        # 둘 다 Unnamed인 경우 인덱스 기반 처리
                        rename_dict[col] = f'col_{i}'
                else:
                    if col in column_mapping:
                        rename_dict[col] = column_mapping[col]
                    else:
                        rename_dict[col] = col
            
            # rename 적용
            df = df.rename(columns=rename_dict)
            
            # Multi-level 컬럼을 단일 레벨로 변환 (rename 후에도 tuple이 남아있을 수 있음)
            new_columns = []
            for col in df.columns:
                if isinstance(col, tuple):
                    # rename_dict에 있으면 사용, 없으면 두 번째 레벨 또는 첫 번째 레벨 사용
                    if col in rename_dict:
                        new_columns.append(rename_dict[col])
                    elif col[1] and not str(col[1]).startswith('Unnamed'):
                        new_columns.append(col[1])
                    elif col[0] and not str(col[0]).startswith('Unnamed'):
                        new_columns.append(col[0])
                    else:
                        new_columns.append(str(col))
                else:
                    new_columns.append(col)
            
            df.columns = new_columns
            
            # Take-Ons의 Att 컬럼 처리 (Passes의 Att와 구분)
            # 원본 파일에서 Take-Ons의 Att는 마지막에서 두 번째 컬럼 근처
            # passes_attempted가 이미 있으면 남은 Att는 take_ons_attempted
            # 중복된 passes_attempted 처리
            if 'passes_attempted' in df.columns:
                # passes_attempted가 두 번 나타나는 경우 (Passes와 Take-Ons 모두)
                cols_list = df.columns.tolist()
                passes_att_indices = [i for i, col in enumerate(cols_list) if col == 'passes_attempted']
                if len(passes_att_indices) == 2:
                    # 두 번째 passes_attempted를 take_ons_attempted로 변경
                    cols_list[passes_att_indices[1]] = 'take_ons_attempted'
                    df.columns = cols_list
                elif 'Att' in df.columns:
                    # Att가 남아있으면 take_ons_attempted로 변경
                    df = df.rename(columns={'Att': 'take_ons_attempted'})
            
            # 첫 번째 행이 헤더 정보인 경우 제거 (league 컬럼 값이 'league'인 행)
            if len(df) > 0 and df.iloc[0, df.columns.get_loc('league')] == 'league':
                df = df.iloc[1:].reset_index(drop=True)
            
            all_leagues_df.append(df)
            print(f"    행 수: {len(df):,}, 컬럼 수: {len(df.columns)}")
        except Exception as e:
            print(f"  ⚠️ {league_file} 읽기 실패: {e}")
            import traceback
            traceback.print_exc()
    else:
        print(f"  ⚠️ 파일을 찾을 수 없습니다: {league_file}")

# 모든 리그 데이터 통합
if all_leagues_df:
    all_stats_df = pd.concat(all_leagues_df, ignore_index=True)
    print(f"\n통합된 리그 데이터: {len(all_stats_df):,} 행")
    print(f"컬럼 수: {len(all_stats_df.columns)}")
    print(f"리그별 행 수:")
    print(all_stats_df['league'].value_counts())
    print(f"\n포함된 컬럼: {sorted(all_stats_df.columns.tolist())}")
else:
    raise ValueError("로드된 리그 데이터가 없습니다.")
vaep_df = pd.read_csv(vaep_file)

# 컬럼명 매핑 (player_id -> playerId)
if 'player_id' in vaep_df.columns:
    vaep_df = vaep_df.rename(columns={'player_id': 'playerId'})
    # 컬럼명 매핑 (기존 컬럼명과 호환)
    if 'vaep_per90' in vaep_df.columns:
        vaep_df = vaep_df.rename(columns={'vaep_per90': 'season_vaep_per90_avg'})
    if 'total_actions' in vaep_df.columns:
        vaep_df = vaep_df.rename(columns={'total_actions': 'num_events'})
    if 'num_games' in vaep_df.columns:
        vaep_df = vaep_df.rename(columns={'num_games': 'matches_played'})
    if 'avg_vaep_per_game' in vaep_df.columns:
        vaep_df = vaep_df.rename(columns={'avg_vaep_per_game': 'season_vaep_per_match'})

# playerId=0 제외 (상황 이벤트 데이터)
print(f"\nVAEP 데이터 원본: {len(vaep_df):,} 행")
if (vaep_df['playerId'] == 0).any():
    zero_count = (vaep_df['playerId'] == 0).sum()
    print(f"playerId=0인 데이터 제외: {zero_count} 행")
    vaep_df = vaep_df[vaep_df['playerId'] != 0].copy()
    print(f"playerId=0 제외 후: {len(vaep_df):,} 행")

print(f"\n통합 리그 데이터: {len(all_stats_df):,} 행")
print(f"VAEP 데이터 (playerId=0 제외): {len(vaep_df):,} 행")
print(f"VAEP 데이터 컬럼: {list(vaep_df.columns)}")

# 선수별 리그 정보 로드
print("\n선수별 리그 정보 로드 중...")

# 1. competitions.json에서 리그 ID -> 이름 매핑 생성
# competitions.json의 이름을 경기 스탯 형식으로 변환
with open(competitions_json, 'r', encoding='utf-8') as f:
    competitions = json.load(f)

# 리그 이름 매핑 (competitions.json -> 경기 스탯 형식)
league_name_mapping = {
    'English first division': 'ENG-Premier League',
    'Spanish first division': 'ESP-La Liga',
    'Italian first division': 'ITA-Serie A',
    'French first division': 'FRA-Ligue 1',
    'German first division': 'GER-Bundesliga',
    'European Championship': 'European Championship',
    'World Cup': 'World Cup'
}

competition_id_to_name = {}
for comp in competitions:
    original_name = comp.get('name', f"Competition {comp['wyId']}")
    # 매핑이 있으면 변환, 없으면 원본 사용
    mapped_name = league_name_mapping.get(original_name, original_name)
    competition_id_to_name[comp['wyId']] = mapped_name

print(f"리그 정보: {len(competition_id_to_name)}개")
print(f"리그 이름 매핑 예시:")
for comp in competitions[:5]:
    original = comp.get('name', 'N/A')
    mapped = competition_id_to_name.get(comp['wyId'], 'N/A')
    if original != mapped:
        print(f"  {original} -> {mapped}")

# 2. matches 파일들에서 game_id -> competitionId 매핑 생성
game_id_to_competition = {}
matches_files = [
    'matches_England.json', 'matches_Spain.json', 'matches_Italy.json',
    'matches_Germany.json', 'matches_France.json',
    'matches_European_Championship.json', 'matches_World_Cup.json'
]

for matches_file in matches_files:
    matches_path = matches_dir / matches_file
    if matches_path.exists():
        with open(matches_path, 'r', encoding='utf-8') as f:
            matches = json.load(f)
        for match in matches:
            game_id = match.get('wyId')
            competition_id = match.get('competitionId')
            if game_id and competition_id:
                game_id_to_competition[int(game_id)] = int(competition_id)
print(f"게임 ID -> 리그 매핑: {len(game_id_to_competition):,}개")

# 3. player_game_vaep_atomic.csv에서 player_id별 game_id 추출
print("player_game_vaep_atomic.csv 로드 중...")
player_game_df = pd.read_csv(player_game_vaep_file)
print(f"게임별 VAEP 데이터: {len(player_game_df):,} 행")

# player_id별로 가장 많이 나타난 리그 찾기
player_league_counts = {}
for _, row in player_game_df.iterrows():
    player_id = int(row['player_id'])
    game_id = int(row['game_id'])
    
    if game_id in game_id_to_competition:
        competition_id = game_id_to_competition[game_id]
        league_name = competition_id_to_name.get(competition_id, f"Unknown {competition_id}")
        
        if player_id not in player_league_counts:
            player_league_counts[player_id] = {}
        if league_name not in player_league_counts[player_id]:
            player_league_counts[player_id][league_name] = 0
        player_league_counts[player_id][league_name] += 1

# 각 선수별로 가장 많이 나타난 리그 선택
player_id_to_league = {}
for player_id, league_counts in player_league_counts.items():
    most_common_league = max(league_counts.items(), key=lambda x: x[1])[0]
    player_id_to_league[player_id] = most_common_league

print(f"선수별 리그 정보: {len(player_id_to_league):,}명")

# 4. vaep_df에 리그 정보 추가
vaep_df['league'] = vaep_df['playerId'].map(player_id_to_league)
print(f"VAEP 데이터에 리그 정보 추가: {vaep_df['league'].notna().sum()}명 / {len(vaep_df)}명")

# 5. 월드컵, 유로 챔피언스 리그 데이터 제외
excluded_leagues = ['World Cup', 'European Championship']
before_filter = len(vaep_df)
vaep_df = vaep_df[~vaep_df['league'].isin(excluded_leagues)].copy()
after_filter = len(vaep_df)
print(f"월드컵/유로 챔피언스 리그 제외: {before_filter - after_filter}명 제외, 남은 데이터: {after_filter}명")

모든 리그 데이터 로드 중...
  로드 중: ENG-Premier League-2017-18.csv
    행 수: 10,448, 컬럼 수: 36
  로드 중: ESP-La Liga-2017-18.csv
    행 수: 10,556, 컬럼 수: 36
  로드 중: FRA-Ligue 1-2017-18.csv
    행 수: 10,576, 컬럼 수: 36
  로드 중: GER-Bundesliga-2017-18.csv
    행 수: 8,512, 컬럼 수: 36
  로드 중: ITA-Serie A-2017-18.csv
    행 수: 10,589, 컬럼 수: 36

통합된 리그 데이터: 50,681 행
컬럼 수: 36
리그별 행 수:
ITA-Serie A           10589
FRA-Ligue 1           10576
ESP-La Liga           10556
ENG-Premier League    10448
GER-Bundesliga         8512
Name: league, dtype: int64

포함된 컬럼: ['GCA', 'SCA', 'age', 'assists', 'blocks', 'carries', 'game', 'game_id', 'goals', 'interceptions', 'jersey_number', 'league', 'minutes', 'nation', 'npxG', 'pass_completion_pct', 'passes_attempted', 'passes_completed', 'penalties', 'penalty_attempts', 'player', 'pos', 'progressive_carries', 'progressive_passes', 'red_cards', 'season', 'shots', 'shots_on_target', 'tackles', 'take_ons_attempted', 'take_ons_successful', 'team', 'touches', 'xAG', 'xG', 'yellow_cards']

## 2. 선수 정보 로드 및 이름 매핑

In [16]:
# players.json 로드
print("\n선수 정보 로드 중...")
with open(players_json, 'r', encoding='utf-8') as f:
    players_data = json.load(f)

# 유니코드 디코딩 함수
def decode_unicode_escapes(text):
    r"""유니코드 이스케이프 시퀀스(\uXXXX)를 실제 문자로 디코딩"""
    if not isinstance(text, str):
        return text
    try:
        if '\\u' in text:
            return text.encode('latin-1').decode('unicode_escape')
        return text
    except:
        return text

# 이름 정규화 함수
def normalize_name(name):
    """이름을 정규화 (소문자, 공백 정리)"""
    if not isinstance(name, str):
        return ""
    name = re.sub(r'\s+', ' ', name.strip().lower())
    return name

# 이름 변형 생성 (이니셜 형식도 처리)
def get_name_variants(first_name, last_name):
    """이름의 여러 변형을 반환 (이니셜 형식도 포함)"""
    variants = []
    
    # 전체 이름
    full = f"{first_name} {last_name}".strip()
    variants.append(normalize_name(full))
    
    # 성을 여러 부분으로 분리
    last_parts = re.split(r'[- ]+', last_name)
    
    # 첫 이름 + 성의 첫 부분
    if len(last_parts) > 0:
        first_last = f"{first_name} {last_parts[0]}".strip()
        variants.append(normalize_name(first_last))
    
    # 첫 이름 + 성의 마지막 부분
    if len(last_parts) > 1:
        first_last = f"{first_name} {last_parts[-1]}".strip()
        variants.append(normalize_name(first_last))
    
    return list(set([v for v in variants if v]))


선수 정보 로드 중...


In [17]:
# 선수 이름 -> ID 매핑 생성
player_name_to_id = {}
player_id_to_name = {}  # ID -> 이름 매핑 (유사도 매칭용)
player_id_to_nation = {}  # ID -> 국적 매핑 (유사도 매칭용)
player_id_to_position = {}  # ID -> 포지션 매핑 (이벤트 데이터에서 추출)

for player in players_data:
    wy_id = player.get('wyId')
    if wy_id is not None:
        wy_id = int(wy_id)
        
        # shortName 처리
        short_name = player.get('shortName', '')
        if short_name:
            short_name = decode_unicode_escapes(short_name)
            normalized_short = normalize_name(short_name)
            if normalized_short:
                player_name_to_id[normalized_short] = wy_id
        
        # firstName + lastName 조합
        first_name = player.get('firstName', '').strip()
        last_name = player.get('lastName', '').strip()
        
        if first_name and last_name:
            # 이름의 여러 변형을 모두 매핑
            variants = get_name_variants(first_name, last_name)
            for variant in variants:
                if variant:
                    player_name_to_id[variant] = wy_id
            
            # ID -> 이름 매핑 생성 (첫 이름 + 성의 첫 부분)
            last_parts = re.split(r'[- ]+', last_name)
            if len(last_parts) > 0:
                first_last = f"{first_name} {last_parts[0]}".strip()
                first_last = decode_unicode_escapes(first_last)
                player_id_to_name[wy_id] = first_last
            else:
                full_name = f"{first_name} {last_name}".strip()
                full_name = decode_unicode_escapes(full_name)
                player_id_to_name[wy_id] = full_name
        else:
            # shortName만 있는 경우
            if short_name:
                player_id_to_name[wy_id] = short_name
        
        # 국적 정보 추출
        birth_area = player.get('birthArea', {})
        if birth_area:
            country_code = birth_area.get('alpha3code', '')
            if country_code:
                player_id_to_nation[wy_id] = country_code
        
        # 포지션 정보 추출 (role에서)
        role = player.get('role', {})
        if role and isinstance(role, dict):
            # code2 사용 (GK, DF, MF, FW)
            position_code = role.get('code2', '')
            if position_code:
                player_id_to_position[wy_id] = position_code
            # code2가 없으면 code3 사용
            elif not position_code:
                position_code3 = role.get('code3', '')
                if position_code3:
                    # code3를 code2로 변환
                    code3_to_code2 = {
                        'GKP': 'GK',
                        'DEF': 'DF',
                        'MID': 'MF',
                        'FWD': 'FW'
                    }
                    player_id_to_position[wy_id] = code3_to_code2.get(position_code3, position_code3)

print(f"선수 매핑: {len(player_name_to_id):,}명")
print(f"ID -> 이름 매핑: {len(player_id_to_name):,}명")
print(f"ID -> 국적 매핑: {len(player_id_to_nation):,}명")
print(f"ID -> 포지션 매핑 (players.json): {len(player_id_to_position):,}명")

# 이벤트 데이터에서 포지션 추론 (players.json에 role이 없는 선수들을 위해)
# 리그 경기만 사용하여 포지션 추론 (월드컵, 유로 등 제외)
print("\n이벤트 데이터에서 포지션 추론 중... (리그 경기만 사용)")

# 리그 경기 competition ID 목록 (월드컵, 유로 제외)
league_competition_ids = {
    364,  # English first division
    795,  # Spanish first division
    524,  # Italian first division
    412,  # French first division
    426   # German first division
}

# game_id_to_competition이 이미 Cell 3에서 생성되었으므로 사용 가능
# 리그 경기만 필터링하기 위해 matchId -> competitionId 매핑 필요
# matches 파일에서 matchId -> competitionId 매핑 생성
match_id_to_competition = {}
matches_files = [
    'matches_England.json', 'matches_Spain.json', 'matches_Italy.json',
    'matches_Germany.json', 'matches_France.json'
]

for matches_file in matches_files:
    matches_path = matches_dir / matches_file
    if not matches_path.exists():
        continue
    
    try:
        with open(matches_path, 'r', encoding='utf-8') as f:
            matches = json.load(f)
        
        for match in matches:
            match_id = match.get('wyId')
            competition_id = match.get('competitionId')
            if match_id and competition_id:
                match_id_to_competition[int(match_id)] = int(competition_id)
    except Exception as e:
        print(f"  경고: {matches_file} 로드 실패: {e}")

print(f"경기 ID -> 리그 매핑: {len(match_id_to_competition):,}개")

# 각 선수의 주 리그 확인을 위해 Cell 3에서 생성된 player_id_to_league를 사용
# 하지만 Cell 6이 Cell 3보다 먼저 실행될 수 있으므로, 
# 일단 모든 리그 경기를 사용하되 월드컵/유로는 제외
# 리그별 competition ID 매핑 (참고용)
league_to_competition_id = {
    'ENG-Premier League': 364,
    'ESP-La Liga': 795,
    'ITA-Serie A': 524,
    'FRA-Ligue 1': 412,
    'GER-Bundesliga': 426
}

# 리그 경기 이벤트만 사용하여 선수별 평균 포지션 수집
# 각 선수의 주 리그 경기만 사용
player_event_positions = {}  # playerId -> {'x': [], 'y': []}

events_files = [
    'events_England.json', 'events_Spain.json', 'events_Italy.json',
    'events_Germany.json', 'events_France.json'
]

for events_file in events_files:
    events_path = matches_dir / events_file
    if not events_path.exists():
        continue
    
    try:
        with open(events_path, 'r', encoding='utf-8') as f:
            events = json.load(f)
        
        league_events_count = 0
        for event in events:
            # matchId 확인하여 리그 경기만 필터링
            match_id = event.get('matchId')
            if match_id and match_id in match_id_to_competition:
                competition_id = match_id_to_competition[match_id]
                # 리그 경기만 사용
                if competition_id in league_competition_ids:
                    if 'playerId' in event and 'positions' in event and event['positions']:
                        player_id = int(event['playerId'])
                        # 첫 번째 포지션 사용 (시작 위치)
                        if len(event['positions']) > 0:
                            pos = event['positions'][0]
                            if 'x' in pos and 'y' in pos:
                                if player_id not in player_event_positions:
                                    player_event_positions[player_id] = {'x': [], 'y': []}
                                player_event_positions[player_id]['x'].append(pos['x'])
                                player_event_positions[player_id]['y'].append(pos['y'])
                                league_events_count += 1
        
        print(f"  {events_file}: 리그 경기 이벤트 {league_events_count:,}개 사용")
    except Exception as e:
        print(f"  경고: {events_file} 로드 실패: {e}")

print(f"리그 경기 이벤트 데이터에서 {len(player_event_positions)}명의 선수 포지션 데이터 수집")

# 평균 포지션 기반으로 포지션 추론
def infer_position_from_coordinates(avg_x, avg_y):
    """x, y 좌표를 기반으로 포지션 추론
    x: 0-100 (왼쪽에서 오른쪽), y: 0-100 (위에서 아래)
    """
    # GK: y < 20
    if avg_y < 20:
        return 'GK'
    # DF: 20 <= y < 40
    elif avg_y < 40:
        if avg_x < 30:
            return 'LB'
        elif avg_x > 70:
            return 'RB'
        else:
            return 'CB'
    # MF: 40 <= y < 60
    elif avg_y < 60:
        if avg_x < 30:
            return 'LM'
        elif avg_x > 70:
            return 'RM'
        else:
            return 'CM'
    # FW: 60 <= y
    else:
        if avg_x < 30:
            return 'LW'
        elif avg_x > 70:
            return 'RW'
        else:
            return 'FW'

# 이벤트 데이터에서 추론한 포지션을 player_id_to_position에 추가
# (players.json에 role이 없는 선수들에 대해서만)
events_inferred_count = 0
for player_id, positions in player_event_positions.items():
    if len(positions['x']) >= 10:  # 최소 10개 이벤트 필요
        avg_x = sum(positions['x']) / len(positions['x'])
        avg_y = sum(positions['y']) / len(positions['y'])
        inferred_pos = infer_position_from_coordinates(avg_x, avg_y)
        
        # players.json에 role이 없는 경우에만 추가
        if player_id not in player_id_to_position:
            player_id_to_position[player_id] = inferred_pos
            events_inferred_count += 1

print(f"이벤트 데이터에서 포지션 추론 완료: {events_inferred_count}명 추가")
print(f"총 포지션 매핑: {len(player_id_to_position):,}명")

선수 매핑: 9,040명
ID -> 이름 매핑: 3,603명
ID -> 국적 매핑: 3,603명
ID -> 포지션 매핑 (players.json): 3,603명

이벤트 데이터에서 포지션 추론 중... (리그 경기만 사용)
경기 ID -> 리그 매핑: 1,826개
  events_England.json: 리그 경기 이벤트 643,150개 사용
  events_Spain.json: 리그 경기 이벤트 628,659개 사용
  events_Italy.json: 리그 경기 이벤트 647,372개 사용
  events_Germany.json: 리그 경기 이벤트 519,407개 사용
  events_France.json: 리그 경기 이벤트 632,807개 사용
리그 경기 이벤트 데이터에서 2569명의 선수 포지션 데이터 수집
이벤트 데이터에서 포지션 추론 완료: 1명 추가
총 포지션 매핑: 3,604명


## 3. 모든 리그 데이터 집계

In [18]:
# 모든 리그 데이터 집계
print("\n모든 리그 데이터 집계 중...")

# 모든 수치형 컬럼 자동 감지 (합계 가능한 컬럼)
numeric_cols = []
for col in all_stats_df.columns:
    if col not in ['league', 'season', 'game', 'team', 'player', 'jersey_number', 'nation', 'pos', 'age', 'game_id']:
        # 수치형 데이터인지 확인
        if all_stats_df[col].dtype in ['int64', 'float64']:
            numeric_cols.append(col)
        # 문자열이지만 숫자로 변환 가능한 경우도 포함 (예: 'Cmp%')
        elif all_stats_df[col].dtype == 'object':
            try:
                # 샘플로 숫자 변환 가능한지 확인
                sample = all_stats_df[col].dropna().head(100)
                if len(sample) > 0:
                    pd.to_numeric(sample, errors='raise')
                    numeric_cols.append(col)
            except:
                pass

print(f"집계할 수치형 컬럼: {len(numeric_cols)}개")
print(f"  - {', '.join(sorted(numeric_cols))}")

agg_dict = {col: 'sum' for col in numeric_cols}

# 경기 수 집계 (game 컬럼을 nunique로 집계)
if 'game' in all_stats_df.columns:
    agg_dict['game'] = 'nunique'

# 리그 정보도 포함 (각 선수가 뛴 리그들)
if 'league' in all_stats_df.columns:
    league_info = all_stats_df.groupby('player')['league'].first().reset_index()

# 팀과 국적 정보도 포함 (가장 많이 뛴 팀과 가장 많이 나타난 국적)
if 'team' in all_stats_df.columns:
    # 각 선수가 가장 많이 뛴 팀 선택
    team_counts = all_stats_df.groupby(['player', 'team']).size().reset_index(name='count')
    team_counts = team_counts.sort_values(['player', 'count'], ascending=[True, False])
    team_counts = team_counts.drop_duplicates('player', keep='first')[['player', 'team']]
    team_counts.columns = ['player', 'team']
    
if 'nation' in all_stats_df.columns:
    # 각 선수의 국적 (첫 번째 값 사용, 대부분 동일)
    nation_info = all_stats_df.groupby('player')['nation'].first().reset_index()

# 포지션 정보 포함 (각 선수가 가장 많이 뛴 포지션 선택)
if 'pos' in all_stats_df.columns:
    # 각 선수가 가장 많이 뛴 포지션 선택
    pos_counts = all_stats_df.groupby(['player', 'pos']).size().reset_index(name='count')
    pos_counts = pos_counts.sort_values(['player', 'count'], ascending=[True, False])
    pos_counts = pos_counts.drop_duplicates('player', keep='first')[['player', 'pos']]
    pos_counts.columns = ['player', 'pos']

# 수치 데이터 집계 (리그별로도 집계)
all_season = all_stats_df.groupby('player').agg(agg_dict).reset_index()

# 컬럼명 정리 (일부 컬럼명만 변경)
column_mapping = {
    'carries': 'carries',  # 이미 매핑됨
    'progressive_carries': 'progressive_carries'  # 이미 매핑됨
}

# 매핑이 필요한 컬럼만 변경
for old_name, new_name in column_mapping.items():
    if old_name in all_season.columns:
        all_season = all_season.rename(columns={old_name: new_name})

# 리그, 팀, 국적, 포지션 정보 병합
if 'league' in all_stats_df.columns:
    all_season = all_season.merge(league_info, on='player', how='left')
if 'team' in all_stats_df.columns:
    all_season = all_season.merge(team_counts, on='player', how='left')
if 'nation' in all_stats_df.columns:
    all_season = all_season.merge(nation_info, on='player', how='left')
if 'pos' in all_stats_df.columns:
    all_season = all_season.merge(pos_counts, on='player', how='left')

# 선수 이름으로 playerId 매핑
all_season['player_normalized'] = all_season['player'].apply(normalize_name)
all_season['playerId'] = all_season['player_normalized'].map(player_name_to_id)

# 포지션 정보 추가 (players.json에서 추출한 포지션)
# 경기 스탯에 포지션이 없거나 비어있는 경우에만 이벤트 데이터에서 가져온 포지션 사용
if 'pos' in all_season.columns:
    # 경기 스탯에 포지션이 없는 경우에만 이벤트 데이터 포지션 사용
    all_season['pos_from_events'] = all_season['playerId'].map(player_id_to_position)
    # 경기 스탯의 포지션이 없거나 비어있으면 이벤트 데이터 포지션으로 채움
    all_season['pos'] = all_season['pos'].fillna(all_season['pos_from_events'])
    all_season = all_season.drop(columns=['pos_from_events'])
else:
    # 경기 스탯에 pos 컬럼이 없으면 이벤트 데이터 포지션 추가
    all_season['pos'] = all_season['playerId'].map(player_id_to_position)

print(f"\n매핑된 선수 수: {all_season['playerId'].notna().sum()} / {len(all_season)}")
if 'team' in all_season.columns:
    print(f"팀 정보 포함: {all_season['team'].notna().sum()} 명")
if 'nation' in all_season.columns:
    print(f"국적 정보 포함: {all_season['nation'].notna().sum()} 명")
if 'pos' in all_season.columns:
    print(f"포지션 정보 포함: {all_season['pos'].notna().sum()} 명")
    print(f"포지션별 선수 수:")
    print(all_season['pos'].value_counts())
if 'league' in all_season.columns:
    print(f"리그 정보 포함: {all_season['league'].notna().sum()} 명")
    print(f"리그별 선수 수:")
    print(all_season['league'].value_counts())

# 필터링: 최소 출전 경기 수 5경기 이상, 경기 시간 220분 이상
print("\n" + "=" * 80)
print("필터링 적용 전 선수 수:", len(all_season))

# 경기 수 확인 (game 컬럼이 nunique로 집계된 경우)
if 'game' in all_season.columns:
    games_col = 'game'
elif 'games' in all_season.columns:
    games_col = 'games'
else:
    games_col = None

# 필터링 조건
filter_conditions = []

# 1. 월드컵, 유로 챔피언스 리그 제외
if 'league' in all_season.columns:
    excluded_leagues = ['World Cup', 'European Championship']
    filter_conditions.append(~all_season['league'].isin(excluded_leagues))
    excluded_count = all_season['league'].isin(excluded_leagues).sum()
    if excluded_count > 0:
        print(f"월드컵/유로 챔피언스 리그 제외: {excluded_count}명")

# 2. 최소 출전 경기 수 5경기 이상
if games_col and games_col in all_season.columns:
    filter_conditions.append(all_season[games_col] >= 5)
    games_below_5 = (all_season[games_col] < 5).sum()
    if games_below_5 > 0:
        print(f"경기 수 5경기 미만 제외: {games_below_5}명")
elif 'matches_played' in all_season.columns:
    # VAEP 데이터와 병합된 경우 matches_played 사용
    filter_conditions.append(all_season['matches_played'] >= 5)
    games_below_5 = (all_season['matches_played'] < 5).sum()
    if games_below_5 > 0:
        print(f"경기 수 5경기 미만 제외: {games_below_5}명")

# 3. 경기 시간 220분 이상
if 'minutes' in all_season.columns:
    filter_conditions.append(all_season['minutes'] >= 220)
    minutes_below_220 = (all_season['minutes'] < 220).sum()
    if minutes_below_220 > 0:
        print(f"경기 시간 220분 미만 제외: {minutes_below_220}명")

# 모든 조건을 만족하는 선수만 선택
if filter_conditions:
    # 모든 조건을 &로 연결
    combined_filter = filter_conditions[0]
    for condition in filter_conditions[1:]:
        combined_filter = combined_filter & condition
    
    before_count = len(all_season)
    all_season = all_season[combined_filter].copy()
    after_count = len(all_season)
    print(f"필터링 적용 후 선수 수: {after_count}")
    print(f"제외된 선수 수: {before_count - after_count}")
else:
    print("필터링 조건이 없습니다.")
print("=" * 80)


모든 리그 데이터 집계 중...
집계할 수치형 컬럼: 26개
  - GCA, SCA, assists, blocks, carries, goals, interceptions, minutes, npxG, pass_completion_pct, passes_attempted, passes_completed, penalties, penalty_attempts, progressive_carries, progressive_passes, red_cards, shots, shots_on_target, tackles, take_ons_attempted, take_ons_successful, touches, xAG, xG, yellow_cards

매핑된 선수 수: 1762 / 2580
팀 정보 포함: 2580 명
국적 정보 포함: 2578 명
포지션 정보 포함: 2579 명
포지션별 선수 수:
CB          461
FW          382
CM          331
GK          202
LB          196
RB          179
DM          144
AM          141
LM          120
LW          119
RW          109
RM          106
WB           55
CM,RM         2
AM,LW         2
RM,RW         2
AM,FW         2
CM,LW         1
LM,RM         1
LM,LW,FW      1
RW,RM         1
CM,DM         1
LB,RB         1
LM,LW         1
CM,CB         1
DM,FW         1
CM,LM         1
FW,MF         1
DM,CM         1
AM,RW         1
AM,WB         1
RB,CM         1
MF            1
LW,LM         1
RW,AM         1


## 3.5. 유사도 기반 선수 매칭 (이름, 국적, 경기 수 비교)

In [19]:
from difflib import SequenceMatcher
import numpy as np

def calculate_name_similarity(name1, name2):
    """이름 유사도 계산 (0-1 사이 값)"""
    if not name1 or not name2:
        return 0.0
    name1_norm = normalize_name(name1)
    name2_norm = normalize_name(name2)
    return SequenceMatcher(None, name1_norm, name2_norm).ratio()

def calculate_nation_match(nation1, nation2):
    """국적 일치 여부 (일치: 1.0, 불일치: 0.0)"""
    if pd.isna(nation1) or pd.isna(nation2):
        return 0.5  # 둘 중 하나라도 없으면 중간 점수
    return 1.0 if str(nation1).strip().upper() == str(nation2).strip().upper() else 0.0

def calculate_games_similarity(games1, games2):
    """경기 수 유사도 계산 (0-1 사이 값)"""
    if pd.isna(games1) or pd.isna(games2):
        return 0.5  # 둘 중 하나라도 없으면 중간 점수
    try:
        g1, g2 = float(games1), float(games2)
        if g1 == 0 and g2 == 0:
            return 1.0
        if g1 == 0 or g2 == 0:
            return 0.0
        # 경기 수 차이를 비율로 계산 (작은 값 기준)
        ratio = min(g1, g2) / max(g1, g2)
        return ratio
    except:
        return 0.5

def calculate_position_similarity(pos1, pos2):
    """포지션 유사도 계산 (0-1 사이 값)"""
    if pd.isna(pos1) or pd.isna(pos2):
        return 0.5  # 둘 중 하나라도 없으면 중간 점수
    
    pos1_str = str(pos1).strip().upper()
    pos2_str = str(pos2).strip().upper()
    
    # 정확히 일치하면 1.0
    if pos1_str == pos2_str:
        return 1.0
    
    # 포지션 코드 분리 (예: "DM,CM" -> ["DM", "CM"])
    pos1_codes = [p.strip() for p in pos1_str.split(',')]
    pos2_codes = [p.strip() for p in pos2_str.split(',')]
    
    # 공통 포지션 코드가 있으면 유사도 계산
    common_codes = set(pos1_codes) & set(pos2_codes)
    if common_codes:
        # 공통 코드가 있으면 0.8 (완전 일치는 아니지만 관련 포지션)
        return 0.8
    
    # 포지션 그룹 정의 (유사한 포지션들)
    position_groups = {
        'GK': ['GK'],
        'DEF': ['CB', 'LB', 'RB', 'LWB', 'RWB', 'SW'],
        'MID': ['DM', 'CM', 'LM', 'RM', 'AM', 'CAM', 'CDM', 'CM', 'LW', 'RW'],
        'FWD': ['CF', 'ST', 'LW', 'RW', 'WF']
    }
    
    # 포지션 그룹 확인
    pos1_groups = set()
    pos2_groups = set()
    
    for code in pos1_codes:
        for group, codes in position_groups.items():
            if code in codes:
                pos1_groups.add(group)
    
    for code in pos2_codes:
        for group, codes in position_groups.items():
            if code in codes:
                pos2_groups.add(group)
    
    # 같은 그룹에 속하면 0.6
    if pos1_groups & pos2_groups:
        return 0.6
    
    # 완전히 다르면 0.0
    return 0.0

def calculate_similarity_score(stats_row, vaep_row, name_weight=0.4, nation_weight=0.25, games_weight=0.15, position_weight=0.2):
    """종합 유사도 점수 계산 (포지션 포함)"""
    name_sim = calculate_name_similarity(stats_row['player'], vaep_row.get('player_name', ''))
    nation_match = calculate_nation_match(stats_row.get('nation'), vaep_row.get('nation'))
    games_sim = calculate_games_similarity(stats_row.get('games'), vaep_row.get('matches_played'))
    pos_sim = calculate_position_similarity(stats_row.get('pos'), vaep_row.get('role'))
    
    total_score = (name_weight * name_sim + 
                   nation_weight * nation_match + 
                   games_weight * games_sim +
                   position_weight * pos_sim)
    
    return total_score, name_sim, nation_match, games_sim, pos_sim

# VAEP 데이터 기준으로 통합 (매핑되지 않은 데이터가 많은 쪽이 VAEP)
# VAEP 데이터에서 선수 이름 정보 준비
vaep_with_names = vaep_df.copy()
vaep_with_names['player_name'] = vaep_with_names['playerId'].map(player_id_to_name)
vaep_with_names['nation'] = vaep_with_names['playerId'].map(player_id_to_nation)

# 매핑되지 않은 VAEP 선수들 (리그 통계 데이터에 없는 선수)
# 이미 매핑된 VAEP 선수 ID 추적
mapped_vaep_ids = set(all_season['playerId'].dropna().astype(int).tolist())
unmapped_vaep = vaep_with_names[~vaep_with_names['playerId'].isin(mapped_vaep_ids)].copy()

# 매핑되지 않은 리그 통계 선수들
unmapped_stats = all_season[all_season['playerId'].isna()].copy()

print(f"\n매핑되지 않은 VAEP 선수: {len(unmapped_vaep)}명")
print(f"매핑되지 않은 리그 통계 선수: {len(unmapped_stats)}명")

# 국적 일치 여부 확인 함수
def nations_match(nation1, nation2):
    """국적이 정확히 일치하는지 확인"""
    if pd.isna(nation1) or pd.isna(nation2):
        return False  # 국적 정보가 없으면 매칭하지 않음
    return str(nation1).strip().upper() == str(nation2).strip().upper()

min_name_similarity_threshold = 0.4  # 최소 이름 유사도 임계값 (국적이 같을 때만 적용)

# 1단계: 국적이 일치하는 경우 먼저 매칭
print("\n" + "=" * 80)
print("1단계: 국적 일치 매칭")
print("=" * 80)

similarity_matches = []
all_candidates = []

# VAEP 데이터 기준으로 매칭 (VAEP → 리그 통계)
for vaep_idx, vaep_row in unmapped_vaep.iterrows():
    vaep_player_id = int(vaep_row['playerId'])
    vaep_nation = vaep_row.get('nation')
    vaep_name = vaep_row.get('player_name', 'N/A')
    
    # VAEP 선수의 국적 정보가 없으면 건너뛰기
    if pd.isna(vaep_nation):
        continue
    
    # 리그 통계 선수 중에서 국적이 같은 선수 찾기
    for stats_idx, stats_row in unmapped_stats.iterrows():
        stats_nation = stats_row.get('nation')
        
        # 국적 정보가 없는 리그 통계 선수는 건너뛰기
        if pd.isna(stats_nation):
            continue
        
        # 1단계: 국적이 같은지 먼저 확인 (필수 조건)
        if not nations_match(vaep_nation, stats_nation):
            continue  # 국적이 다르면 매칭하지 않음
        
        # 2단계: 국적이 같으면 이름 유사도 계산
        name_sim = calculate_name_similarity(vaep_name, stats_row['player'])
        
        # 이름 유사도가 임계값 이상인 경우만 후보에 추가
        if name_sim >= min_name_similarity_threshold:
            # 경기 수 유사도 계산
            games_sim = calculate_games_similarity(vaep_row.get('matches_played'), stats_row.get('games'))
            # 포지션 유사도 계산
            pos_sim = calculate_position_similarity(stats_row.get('pos'), vaep_row.get('role'))
            
            # 종합 점수 계산 (국적은 이미 일치하므로 1.0, 이름, 경기 수, 포지션 고려)
            # 국적이 같으므로 이름 유사도에 더 높은 가중치 부여
            total_score = 0.70 * name_sim + 0.15 * games_sim + 0.15 * pos_sim  # 이름 70%, 경기 수 15%, 포지션 15%
            
            all_candidates.append({
                'vaep_player_id': vaep_player_id,
                'vaep_name': vaep_name,
                'vaep_nation': vaep_nation,
                'vaep_games': vaep_row.get('matches_played'),
                'vaep_pos': vaep_row.get('role'),
                'stats_idx': stats_idx,
                'stats_player': stats_row['player'],
                'stats_nation': stats_row.get('nation'),
                'stats_games': stats_row.get('games'),
                'stats_pos': stats_row.get('pos'),
                'total_score': total_score,
                'name_sim': name_sim,
                'nation_match': 1.0,  # 국적이 같으므로 항상 1.0
                'games_sim': games_sim,
                'pos_sim': pos_sim
            })

# 점수 순으로 정렬 (높은 점수 우선)
all_candidates.sort(key=lambda x: x['total_score'], reverse=True)

print(f"\n국적 일치 후보 수: {len(all_candidates)}개")
if len(all_candidates) > 0:
    print(f"  - 국적 일치 조건을 통과한 후보 중 이름 유사도 {min_name_similarity_threshold} 이상: {len(all_candidates)}개")

# 중복 없이 최적 매칭 선택
selected_stats_indices = set()
selected_vaep_ids = set()

for candidate in all_candidates:
    stats_idx = candidate['stats_idx']
    vaep_id = candidate['vaep_player_id']
    
    # 이미 매칭된 리그 통계 선수나 VAEP 선수는 건너뛰기
    if stats_idx in selected_stats_indices or vaep_id in selected_vaep_ids:
        continue
    
    # 매칭 추가
    selected_stats_indices.add(stats_idx)
    selected_vaep_ids.add(vaep_id)
    similarity_matches.append({
        'vaep_player_id': vaep_id,
        'vaep_name': candidate['vaep_name'],
        'vaep_nation': candidate['vaep_nation'],
        'vaep_games': candidate['vaep_games'],
        'vaep_pos': candidate.get('vaep_pos'),
        'stats_player': candidate['stats_player'],
        'stats_nation': candidate['stats_nation'],
        'stats_games': candidate['stats_games'],
        'stats_pos': candidate.get('stats_pos'),
        'matched_playerId': vaep_id,  # all_season 업데이트용
        'matched_name': candidate['vaep_name'],
        'matched_nation': candidate['vaep_nation'],
        'matched_games': candidate['vaep_games'],
        'total_score': candidate['total_score'],
        'name_sim': candidate['name_sim'],
        'nation_match': candidate['nation_match'],
        'games_sim': candidate['games_sim'],
        'pos_sim': candidate.get('pos_sim', 0.5)
    })

print(f"\n1단계 국적 일치 매칭 완료: {len(similarity_matches)}명")

# 2단계: 남은 VAEP 선수들을 리그 통계 선수와 통합 (국적이 다른 경우도 포함)
print("\n" + "=" * 80)
print("2단계: 남은 VAEP 선수 통합")
print("=" * 80)

# 1단계에서 매칭된 VAEP 선수 ID
matched_vaep_ids_step1 = set([m['vaep_player_id'] for m in similarity_matches])
remaining_vaep = unmapped_vaep[~unmapped_vaep['playerId'].isin(matched_vaep_ids_step1)].copy()

# 1단계에서 매칭된 리그 통계 선수 인덱스
matched_stats_indices_step1 = set([m.get('stats_idx', None) for m in similarity_matches if 'stats_idx' in m])
# stats_idx가 없는 경우를 위해 stats_player로 찾기
matched_stats_players_step1 = set([m['stats_player'] for m in similarity_matches])
remaining_stats = unmapped_stats[~unmapped_stats['player'].isin(matched_stats_players_step1)].copy()

print(f"남은 VAEP 선수: {len(remaining_vaep)}명")
print(f"남은 리그 통계 선수: {len(remaining_stats)}명")

# 남은 VAEP 선수들을 리그 통계 선수와 통합 (국적 조건 없이 이름 유사도만으로)
if len(remaining_vaep) > 0 and len(remaining_stats) > 0:
    remaining_candidates = []
    
    for vaep_idx, vaep_row in remaining_vaep.iterrows():
        vaep_player_id = int(vaep_row['playerId'])
        vaep_name = vaep_row.get('player_name', 'N/A')
        vaep_nation = vaep_row.get('nation')
        
        for stats_idx, stats_row in remaining_stats.iterrows():
            stats_name = stats_row['player']
            stats_nation = stats_row.get('nation')
            
            # 이름 유사도 계산 (국적 조건 없이)
            name_sim = calculate_name_similarity(vaep_name, stats_name)
            
            # 이름 유사도가 임계값 이상인 경우 후보에 추가
            if name_sim >= min_name_similarity_threshold:
                # 국적 일치 여부 확인
                nation_match = 1.0 if nations_match(vaep_nation, stats_nation) else 0.0
                
                # 경기 수 유사도 계산
                games_sim = calculate_games_similarity(vaep_row.get('matches_played'), stats_row.get('games'))
                
                # 종합 점수 계산 (국적이 일치하면 가중치 높임)
                # 이름 유사도를 많이 반영
                if nation_match == 1.0:
                    total_score = 0.85 * name_sim + 0.15 * games_sim  # 이름 85%, 경기 수 15%
                else:
                    total_score = 0.75 * name_sim + 0.25 * games_sim  # 이름 75%, 경기 수 25% (국적이 다르면 점수 낮춤)
                
                remaining_candidates.append({
                    'vaep_player_id': vaep_player_id,
                    'vaep_name': vaep_name,
                    'vaep_nation': vaep_nation,
                    'vaep_games': vaep_row.get('matches_played'),
                    'stats_idx': stats_idx,
                    'stats_player': stats_name,
                    'stats_nation': stats_nation,
                    'stats_games': stats_row.get('games'),
                    'total_score': total_score,
                    'name_sim': name_sim,
                    'nation_match': nation_match,
                    'games_sim': games_sim
                })
    
    # 점수 순으로 정렬
    remaining_candidates.sort(key=lambda x: x['total_score'], reverse=True)
    
    # 중복 없이 매칭 선택
    remaining_selected_stats = set()
    remaining_selected_vaep = set()
    
    for candidate in remaining_candidates:
        stats_idx = candidate['stats_idx']
        vaep_id = candidate['vaep_player_id']
        
        if stats_idx in remaining_selected_stats or vaep_id in remaining_selected_vaep:
            continue
        
        remaining_selected_stats.add(stats_idx)
        remaining_selected_vaep.add(vaep_id)
        similarity_matches.append({
            'vaep_player_id': vaep_id,
            'vaep_name': candidate['vaep_name'],
            'vaep_nation': candidate['vaep_nation'],
            'vaep_games': candidate['vaep_games'],
            'vaep_pos': candidate.get('vaep_pos'),
            'stats_player': candidate['stats_player'],
            'stats_nation': candidate['stats_nation'],
            'stats_games': candidate['stats_games'],
            'stats_pos': candidate.get('stats_pos'),
            'matched_playerId': vaep_id,
            'matched_name': candidate['vaep_name'],
            'matched_nation': candidate['vaep_nation'],
            'matched_games': candidate['vaep_games'],
            'total_score': candidate['total_score'],
            'name_sim': candidate['name_sim'],
            'nation_match': candidate['nation_match'],
            'games_sim': candidate['games_sim'],
            'pos_sim': candidate.get('pos_sim', 0.5)
        })
    
    print(f"2단계 통합 완료: {len(remaining_selected_vaep)}명")

print(f"\n총 유사도 기반 매칭 발견: {len(similarity_matches)}명")

# 중복 매칭 확인 및 분석
if similarity_matches:
    match_df = pd.DataFrame(similarity_matches)
    
    # 중복된 VAEP playerId 확인
    vaep_id_counts = match_df['matched_playerId'].value_counts()
    duplicate_vaep = vaep_id_counts[vaep_id_counts > 1]
    
    # 중복된 리그 통계 선수 이름 확인
    stats_name_counts = match_df['stats_player'].value_counts()
    duplicate_stats = stats_name_counts[stats_name_counts > 1]
    
    if len(duplicate_vaep) > 0:
        print(f"\n⚠️ 경고: 중복된 VAEP playerId 발견 ({len(duplicate_vaep)}개):")
        for vaep_id, count in duplicate_vaep.items():
            vaep_name = player_id_to_name.get(int(vaep_id), f"playerId: {vaep_id}")
            matched_stats = match_df[match_df['matched_playerId'] == vaep_id]['stats_player'].tolist()
            print(f"  - VAEP: {vaep_name} (playerId: {vaep_id}) -> {count}명의 리그 통계 선수와 매칭")
            for stats_name in matched_stats:
                match_info = match_df[(match_df['matched_playerId'] == vaep_id) & (match_df['stats_player'] == stats_name)].iloc[0]
                print(f"    * {stats_name} (점수: {match_info['total_score']:.4f}, 이름유사도: {match_info['name_sim']:.4f}, 국적일치: {match_info['nation_match']:.1f}, 경기유사도: {match_info['games_sim']:.4f})")
    
    if len(duplicate_stats) > 0:
        print(f"\n⚠️ 경고: 중복된 리그 통계 선수 이름 발견 ({len(duplicate_stats)}개):")
        for stats_name, count in duplicate_stats.items():
            matched_vaeps = match_df[match_df['stats_player'] == stats_name]['matched_playerId'].tolist()
            print(f"  - 리그 통계: {stats_name} -> {count}명의 VAEP 선수와 매칭")
            for vaep_id in matched_vaeps:
                vaep_name = player_id_to_name.get(int(vaep_id), f"playerId: {vaep_id}")
                match_info = match_df[(match_df['stats_player'] == stats_name) & (match_df['matched_playerId'] == vaep_id)].iloc[0]
                print(f"    * VAEP: {vaep_name} (playerId: {vaep_id}, 점수: {match_info['total_score']:.4f})")
    
    # 중복 제거: 각 VAEP playerId와 리그 통계 선수 이름당 가장 높은 점수의 매칭만 유지
    if len(duplicate_vaep) > 0 or len(duplicate_stats) > 0:
        print(f"\n중복 제거 중...")
        # VAEP playerId 기준으로 최고 점수만 유지
        match_df_dedup = match_df.sort_values('total_score', ascending=False).drop_duplicates('matched_playerId', keep='first')
        # 리그 통계 선수 이름 기준으로도 최고 점수만 유지
        match_df_dedup = match_df_dedup.sort_values('total_score', ascending=False).drop_duplicates('stats_player', keep='first')
        
        removed_count = len(similarity_matches) - len(match_df_dedup)
        if removed_count > 0:
            print(f"중복 제거: {removed_count}개의 중복 매칭 제거됨")
            similarity_matches = match_df_dedup.to_dict('records')

# 매칭 결과를 all_season에 반영
if similarity_matches:
    match_df = pd.DataFrame(similarity_matches)
    
    # 상세 정보 출력
    # 모든 매칭 점수가 1.0이면 보여주지 않고, 그 이하면 보여줌
    all_perfect = (match_df['total_score'] == 1.0).all()
    
    if all_perfect:
        print(f"\n매칭 결과: {len(match_df)}명 (모든 매칭이 완벽하므로 상세 정보 생략)")
    else:
        # 완벽하지 않은 매칭만 필터링
        imperfect_matches = match_df[match_df['total_score'] < 1.0].copy()
        
        if len(imperfect_matches) > 0:
            print(f"\n매칭 결과 (완벽하지 않은 매칭 {len(imperfect_matches)}명):")
            display_cols = ['vaep_name', 'stats_player', 'vaep_nation', 'stats_nation', 
                            'vaep_games', 'stats_games', 'total_score', 'name_sim', 'nation_match', 'games_sim']
            print(imperfect_matches.sort_values('total_score', ascending=False)[display_cols].to_string(index=False))
        else:
            print(f"\n매칭 결과: {len(match_df)}명 (모든 매칭이 완벽함)")
    
    # all_season에 playerId 업데이트
    for match in similarity_matches:
        all_season.loc[
            all_season['player'] == match['stats_player'], 
            'playerId'
        ] = match['matched_playerId']
    
    print(f"\n업데이트된 매핑된 선수 수: {all_season['playerId'].notna().sum()} / {len(all_season)}")
else:
    print("\n유사도 기반 매칭을 찾지 못했습니다.")


매핑되지 않은 VAEP 선수: 1111명
매핑되지 않은 리그 통계 선수: 658명

1단계: 국적 일치 매칭

국적 일치 후보 수: 4094개
  - 국적 일치 조건을 통과한 후보 중 이름 유사도 0.4 이상: 4094개

1단계 국적 일치 매칭 완료: 461명

2단계: 남은 VAEP 선수 통합
남은 VAEP 선수: 650명
남은 리그 통계 선수: 197명
2단계 통합 완료: 197명

총 유사도 기반 매칭 발견: 658명

매칭 결과 (완벽하지 않은 매칭 658명):
                       vaep_name               stats_player vaep_nation stats_nation  vaep_games stats_games  total_score  name_sim  nation_match  games_sim
                  Jérôme Gondorf             Jérôme Gondorf         DEU          GER          20        None     0.875000  1.000000           0.0        0.5
                     Gaël Kakuta                Gaël Kakuta         FRA          COD          36        None     0.875000  1.000000           0.0        0.5
                   Kévin Malcuit              Kévin Malcuit         FRA          MAR          23        None     0.875000  1.000000           0.0        0.5
                 Danijel Subašić            Danijel Subašić         HRV          CRO          44        N

## 4. VAEP 데이터와 병합

In [20]:
# VAEP 데이터와 병합 (outer join으로 양쪽 데이터 모두 유지)
print("\n데이터 병합 중...")

merged_df = all_season.merge(vaep_df, on='playerId', how='outer', suffixes=('', '_vaep'))

# 리그 정보 병합 (리그 통계 우선, 없으면 VAEP 데이터의 리그 사용)
if 'league_vaep' in merged_df.columns:
    merged_df['league'] = merged_df['league'].fillna(merged_df['league_vaep'])
    merged_df = merged_df.drop(columns=['league_vaep'])
elif 'league' not in merged_df.columns and 'league_vaep' in merged_df.columns:
    merged_df['league'] = merged_df['league_vaep']
    merged_df = merged_df.drop(columns=['league_vaep'])

print(f"병합된 데이터: {len(merged_df):,} 행")
print(f"리그 정보 포함: {merged_df['league'].notna().sum()}명 / {len(merged_df)}명")


데이터 병합 중...
병합된 데이터: 2,559 행
리그 정보 포함: 2559명 / 2559명


## 5. 선수 이름 설정 (Premier League 이름 우선)

In [21]:
# players.json에서 선수 이름 및 국적 가져오기 (VAEP 데이터에만 있는 선수용)
player_id_to_name = {}
player_id_to_nation = {}

for player in players_data:
    wy_id = player.get('wyId')
    if wy_id is not None:
        wy_id = int(wy_id)
        first_name = player.get('firstName', '').strip()
        last_name = player.get('lastName', '').strip()
        if first_name and last_name:
            # 첫 이름 + 성의 첫 부분 (Premier League 형식과 일치)
            last_parts = re.split(r'[- ]+', last_name)
            if len(last_parts) > 0:
                first_last = f"{first_name} {last_parts[0]}".strip()
                first_last = decode_unicode_escapes(first_last)
                player_id_to_name[wy_id] = first_last
            else:
                full_name = f"{first_name} {last_name}".strip()
                full_name = decode_unicode_escapes(full_name)
                player_id_to_name[wy_id] = full_name
        else:
            short_name = player.get('shortName', '')
            if short_name:
                short_name = decode_unicode_escapes(short_name)
                player_id_to_name[wy_id] = short_name
        
        # 국적 정보 추출
        birth_area = player.get('birthArea', {})
        if birth_area:
            country_code = birth_area.get('alpha3code', '')
            if country_code:
                player_id_to_nation[wy_id] = country_code

# 이름 설정: Premier League 이름 우선, 없으면 players.json에서 찾기
# Premier League 데이터에 있는 선수는 항상 Premier League 이름 사용
merged_df['player_name'] = merged_df['player'].fillna(
    merged_df['playerId'].map(player_id_to_name).fillna('Unknown')
)

# 국적 설정: Premier League 데이터 우선, 없으면 players.json에서 찾기
if 'nation' not in merged_df.columns:
    merged_df['nation'] = None
merged_df['nation'] = merged_df['nation'].fillna(
    merged_df['playerId'].map(player_id_to_nation)
)

print(f"\n이름 설정 완료")
print(f"Premier League 이름 사용: {(merged_df['player'].notna()).sum()}명")
print(f"players.json 이름 사용: {((merged_df['player'].isna()) & (merged_df['playerId'].notna())).sum()}명")
print(f"Unknown: {(merged_df['player_name'] == 'Unknown').sum()}명")
print(f"\n국적 정보:")
print(f"Premier League 국적 사용: {merged_df['nation'].notna().sum()}명")


이름 설정 완료
Premier League 이름 사용: 2106명
players.json 이름 사용: 453명
Unknown: 0명

국적 정보:
Premier League 국적 사용: 2559명


## 6. 최종 데이터 정리 및 저장

In [22]:
# playerId=0 제외 확인 (혹시 모를 경우를 대비)
if (merged_df['playerId'] == 0).any():
    print(f"\nplayerId=0인 데이터 제외: {(merged_df['playerId'] == 0).sum()} 행")
    merged_df = merged_df[merged_df['playerId'] != 0].copy()

# 이벤트 하나당 VAEP 계산 (season_vaep_total / num_events)
if 'season_vaep_total' in merged_df.columns and 'num_events' in merged_df.columns:
    import numpy as np
    merged_df['season_vaep_per_event'] = merged_df.apply(
        lambda row: row['season_vaep_total'] / row['num_events'] 
        if pd.notna(row['season_vaep_total']) and pd.notna(row['num_events']) and row['num_events'] > 0 
        else np.nan, 
        axis=1
    )
    print(f"\n이벤트당 VAEP 계산 완료")
    print(f"  - 계산된 선수 수: {merged_df['season_vaep_per_event'].notna().sum()}명")
    if merged_df['season_vaep_per_event'].notna().sum() > 0:
        print(f"  - 평균 이벤트당 VAEP: {merged_df['season_vaep_per_event'].mean():.6f}")

# 최종 컬럼 목록 (모든 통계 포함)
# 기본 정보 먼저, 그 다음 모든 통계, 마지막에 VAEP 지표
base_columns = ['playerId', 'player_name', 'player', 'team', 'nation', 'league', 'pos', 'games', 'matches_played']
vaep_columns = ['season_vaep_total', 'season_vaep_per90_avg', 'season_vaep_per_match', 'season_vaep_per_event', 'num_events', 'role']

# merged_df에서 사용 가능한 모든 통계 컬럼 가져오기 (VAEP 컬럼 제외)
stats_columns = [col for col in merged_df.columns 
                 if col not in base_columns + vaep_columns 
                 and col not in ['player_normalized']]

# 정렬: 기본 정보 -> 통계 (알파벳 순) -> VAEP
final_columns = base_columns + sorted(stats_columns) + vaep_columns

# 존재하는 컬럼만 선택
available_columns = [col for col in final_columns if col in merged_df.columns]
merged_df = merged_df[available_columns]

# 정렬 (VAEP per 90 기준 내림차순, NaN은 마지막에)
if 'season_vaep_per90_avg' in merged_df.columns:
    merged_df = merged_df.sort_values('season_vaep_per90_avg', ascending=False, na_position='last')

# 저장
print(f"\n결과 저장 중: {output_file}")
merged_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"\n통합 완료!")
print(f"총 선수 수: {len(merged_df):,}")
print(f"VAEP 데이터가 있는 선수: {merged_df['season_vaep_per90_avg'].notna().sum():,}")
print(f"Premier League 통계가 있는 선수: {merged_df['xG'].notna().sum() if 'goals' in merged_df.columns else 0:,}")
both_data = ((merged_df['season_vaep_per90_avg'].notna()) & (merged_df['xG'].notna())).sum() if 'goals' in merged_df.columns else 0
print(f"양쪽 데이터가 모두 있는 선수: {both_data:,}")

unknown_count = (merged_df['player_name'] == 'Unknown').sum()
print(f"Unknown 선수 수: {unknown_count}")


이벤트당 VAEP 계산 완료
  - 계산된 선수 수: 2554명
  - 평균 이벤트당 VAEP: 0.006809

결과 저장 중: ../data/player_season_stats_with_vaep_all_leagues_2017-18.csv

통합 완료!
총 선수 수: 2,559
VAEP 데이터가 있는 선수: 2,554
Premier League 통계가 있는 선수: 2,106
양쪽 데이터가 모두 있는 선수: 2,101
Unknown 선수 수: 0
